In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.safetensors:  53%|#####2    | 1.63G/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [5]:
from threading import Thread
from transformers import TextIteratorStreamer

MAX_INPUT_TOKEN_LENGTH = 4096
system_prompt = "translate to Chinese"
chat_history = []
message = """
The origins of the American Civil War were rooted in the desire of the Southern states to preserve the institution of slavery.[1] Historians in the 21st century overwhelmingly agree on the centrality of slavery in the conflict. They disagree on which aspects (ideological, economic, political, or social) were most important, and on the North's reasons for refusing to allow the Southern states to secede.[2] The pseudo-historical Lost Cause ideology denies that slavery was the principal cause of the secession, a view disproven by historical evidence, notably some of the seceding states' own secession documents.[3] After leaving the Union, Mississippi issued a declaration stating, "Our position is thoroughly identified with the institution of slavery—the greatest material interest of the world."[4][5]

The principal political battle leading to Southern secession was over whether slavery would expand into the Western territories destined to become states. Initially Congress had admitted new states into the Union in pairs, one slave and one free. This had kept a sectional balance in the Senate but not in the House of Representatives, as free states outstripped slave states in numbers of eligible voters.[6] Thus, at mid-19th century, the free-versus-slave status of the new territories was a critical issue, both for the North, where anti-slavery sentiment had grown, and for the South, where the fear of slavery's abolition had grown. Another factor leading to secession and the formation of the Confederacy was the development of white Southern nationalism in the preceding decades.[7] The primary reason for the North to reject secession was to preserve the Union, a cause based on American nationalism.[8]

Abraham Lincoln won the 1860 presidential election. His victory triggered declarations of secession by seven slave states of the Deep South, all of whose riverfront or coastal economies were based on cotton that was cultivated by slave labor. They formed the Confederate States of America after Lincoln was elected in November 1860 but before he took office in March 1861. Nationalists in the North and "Unionists" in the South refused to accept the declarations of secession. No foreign government ever recognized the Confederacy. The U.S. government, under President James Buchanan, refused to relinquish its forts that were in territory claimed by the Confederacy. The war itself began on April 12, 1861, when Confederate forces bombarded the Union's Fort Sumter, in the harbor of Charleston, South Carolina.

Background factors in the run up to the Civil War were partisan politics, abolitionism, nullification versus secession, Southern and Northern nationalism, expansionism, economics, and modernization in the antebellum period. As a panel of historians emphasized in 2011, "while slavery and its various and multifaceted discontents were the primary cause of disunion, it was disunion itself that sparked the war."[9]
"""

conversation = []
if system_prompt:
    conversation.append({"role": "system", "content": system_prompt})
conversation += chat_history
conversation.append({"role": "user", "content": message})

input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt")
if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
    input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
    gr.Warning(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")
input_ids = input_ids.to(model.device)

streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=False, skip_special_tokens=False)
generate_kwargs = dict(
    {"input_ids": input_ids},
    streamer=streamer,
    max_new_tokens=1024,
    do_sample=True,
    top_p=0.8,
    top_k=10,
    temperature=0.2,
    num_beams=1,
    repetition_penalty=1.2,
)

t = Thread(target=model.generate, kwargs=generate_kwargs)
t.start()

try:
    outputs = []
    for text in streamer:
        outputs.append(text)
        print(text, end="", flush=True)
finally:
    t.join()

<|im_start|>system
translate to Chinese<|im_end|>
<|im_start|>user

The origins of the American Civil War were rooted in the desire of the Southern states to preserve the institution of slavery.[1] Historians in the 21st century overwhelmingly agree on the centrality of slavery in the conflict. They disagree on which aspects (ideological, economic, political, or social) were most important, and on the North's reasons for refusing to allow the Southern states to secede.[2] The pseudo-historical Lost Cause ideology denies that slavery was the principal cause of the secession, a view disproven by historical evidence, notably some of the seceding states' own secession documents.[3] After leaving the Union, Mississippi issued a declaration stating, "Our position is thoroughly identified with the institution of slavery—the greatest material interest of the world."[4][5]

The principal political battle leading to Southern secession was over whether slavery would expand into the Western territ